In [3]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
import torch
import os

In [2]:
from tqdm import tqdm
split_num = 4500
load_file = './total_detail_0_4500.json'
count = 0
with open('../total_detail.json','r') as f:
    for i in tqdm(f):
        with open(load_file,'a+') as f1:
            f1.write(i)
            count += 1
        if (count + 1) % split_num == 0:
            index_ = int((count+1)//split_num)
            load_file = './total_detail_'+str(index_*split_num)+'_' + str((index_+1)*split_num) + '.json'
            print(load_file)

4517it [00:38, 133.07it/s]

./total_detail_4500_9000.json


9014it [01:18, 124.43it/s]

./total_detail_9000_13500.json


13521it [01:56, 121.96it/s]

./total_detail_13500_18000.json


18021it [02:35, 123.84it/s]

./total_detail_18000_22500.json


18167it [02:37, 115.65it/s]


In [ ]:
import json
with open('./total_detail_0_9000.json','r') as f:
    for i in tqdm(f):
        app = json.loads(i)

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
device = torch.device("cuda:1")
print("Using {} device".format(device))

Using cuda:1 device


In [8]:
"cuda:%s"%1

'cuda:1'

In [7]:
back_translation_aug = naw.BackTranslationAug(from_model_name='facebook/wmt19-en-de',
                                              to_model_name='facebook/wmt19-de-en',
                                              device=device,
                                              max_length=512)

In [4]:
import json
import random
from tqdm import tqdm

In [5]:
batch_size = 64
raw_text_saved = []
clean_text_saved = []
voted_up_saved = []
sentiment_score_saved = []
save_count = 0
with open('../total_detail.json','r') as f:
    for i in tqdm(f):
        app = json.loads(i)
        info = app['game_info']
        reviews = info['reviews']
        for review in tqdm(reviews):
            clean_text = review['sentiment_text']
            raw_text = review['review']
            if clean_text:
                sentiment_score = review['sentiment_score']
                voted_up = review['voted_up']
                raw_text_saved.append(raw_text)
                clean_text_saved.append(clean_text)
                voted_up_saved.append(voted_up)
                sentiment_score_saved.append(sentiment_score)
                save_count += 1
                if save_count == batch_size:
                    augmented_text = back_translation_aug.augment(clean_text_saved)
                    assert len(augmented_text) == batch_size
                    with open('./review_uda_data.json','a+') as f1:
                        for a,b,c,d,e in zip(raw_text_saved,
                                             clean_text_saved,
                                             augmented_text,
                                             voted_up_saved,
                                             sentiment_score_saved):
                            item = {'raw_text':a,
                                    'clean_text':b,
                                    'augmented_text':c,
                                    'voted_up':d,
                                    'sentiment_score':e}
                            f1.write(json.dumps(item) + '\n')
                    raw_text_saved = []
                    clean_text_saved = []
                    voted_up_saved = []
                    sentiment_score_saved = []
                    save_count = 0

0it [00:00, ?it/s]
100%|██████████| 31/31 [00:00<00:00, 170187.73it/s]

 17%|█▋        | 32/193 [00:05<00:27,  5.96it/s]
1it [00:05,  5.38s/it]


AttributeError: 'DataParallel' object has no attribute 'generate'